# AgentKit Observability Tutorial 02: 非 Agentkit Runtime 部署的 Agent 开启可观测

## 概述

本教程面向 **第一次接触 AgentKit SDK 的新手用户**，在一个 Jupyter Notebook 中，带你完成：
  - 对于未在 agentkit runtime 中部署的 Agentit runtime 实例，如本地部署等，如何完成可观测数据的接入与上报。
  - 如何查看 Agentkit 可观测数据。

你可以直接从上到下依次运行每个代码单元。建议在 **仓库根目录** 或已安装好 `agentkit` 的 Python 环境中运行。

### Tutorial 说明
|分类| 详情                                              |
|:--------------------|:------------------------------------------------|
| Agent 类型          | 单Agent                                          |
| Agent 开发框架   | Volcengine Agent Development Kit(VeADK)         |
| Tutorial 核心模块 | AgentKit Runtime, ApmPlus                       |
| 教程复杂度          | 简单                                              |
| 使用到的SDK            | AgentKit Python SDK, VeADK |


## 教程


### 0. 前置条件
要执行本教程，您需要：
1. **火山引擎账号 AK SK**：一个已开通 AgentKit 与 APMPlus 服务的火山引擎账号，并获取其 AccessKey & SecretKey。并将其配置到环境变量中。
    >**提示**：请确保您的 AccessKey 拥有 `AgentKitFullAccess` 与 `APMPlusFullAccess` 权限。

2. **环境准备**：Python 3.10+ 环境准备并安装相关依赖



### 1. 准备工作
依赖安装。安装 agentkit sdk 等依赖。

In [ ]:
!pip install -r requirements.txt

### 2. 初始化 agentkit 项目
初始化一个 agentkit 项目，可基于[AgentKit Runtime 教程](https://volcengine.github.io/agentkit-sdk-python/content/4.runtime/1.runtime_quickstart.html) 创建一个 Agent。可通过执行以下命令初始化项目：

In [ ]:
!agentkit init

初始化完成后，会自动生成 simple_agent.py 代码。在您的 Agent 代码中引入 APMPlusExporter，从而将 Agent 运行时的指标数据导出到 APMPlus 服务并进行观测。
以下为代码示例：

In [ ]:
# Copyright (c) 2025 Beijing Volcano Engine Technology Co., Ltd. and/or its affiliates.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging

from veadk import Agent, Runner

from agentkit.apps import AgentkitSimpleApp
from veadk.prompts.agent_default_prompt import DEFAULT_DESCRIPTION, DEFAULT_INSTRUCTION
# 引入相关依赖包
from veadk.tracing.telemetry.exporters.apmplus_exporter import APMPlusExporter
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


app = AgentkitSimpleApp()

agent_name = "Agent"
description = DEFAULT_DESCRIPTION 
system_prompt = DEFAULT_INSTRUCTION 


tools = []

# from veadk.tools.builtin_tools.web_search import web_search
# tools.append(web_search)

# 初始化 APMPlusExporter 实例
exporters = [
    APMPlusExporter() 
]
tracer = OpentelemetryTracer(
    exporters = exporters
)

agent = Agent(
    name=agent_name,
    description=description,
    instruction=system_prompt,
    tools=tools,
    tracers=[tracer], # 设置 Agent Tracer 参数
)
runner = Runner(agent=agent)


@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    user_id = headers["user_id"]
    session_id = headers["session_id"]

    logger.info(
        f"Running agent with prompt: {prompt}, user_id: {user_id}, session_id: {session_id}"
    )
    response = await runner.run(messages=prompt, user_id=user_id, session_id=session_id)

    logger.info(f"Run response: {response}")
    return response


@app.ping
def ping() -> str:
    return "pong!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


## 3. 开启 APMPlus 可观测

对于未在 agentkit runtime 中部署的 Agentit runtime 实例，在代码里引入 APMPlusExporter 后，您仍需要配置以下环境变量以完成可观测数据的产生与上报。

### 环境变量列表：

OBSERVABILITY_OPENTELEMETRY_APMPLUS_API_KEY: APMPlus 服务的 API Key 
OBSERVABILITY_OPENTELEMETRY_APMPLUS_ENDPOINT: APMPlus 服务的 Endpoint, 例如 http://apmplus-cn-beijing.volces.com:4317
OBSERVABILITY_OPENTELEMETRY_APMPLUS_SERVICE_NAME: APMPlus 的 Service Name, 例如 simple_agent

APMPlus API Key 和 Endpoint 可以在 APMPlus 控制台中获取。参考 https://www.volcengine.com/docs/6431/147492?lang=zh。

如未配置这些环境，则使用当前火山账号的 APIKey, Endpoint 使用 APMPlus cn-beijing 的 EndPoint，即 http://apmplus-cn-beijing.volces.com:4317。




### 4. 运行 Agent
在完成 Agent 代码编写和配置后，您可以运行以下命令启动 Agent 服务：

In [ ]:
!python simple_agent.py

服务将监听 http://0.0.0.0:8000。现在，打开另一个终端，使用 curl 来测试您的 Agent：


In [ ]:
!curl --location 'http://localhost:8000/invoke' \
  --header 'Content-Type: application/json' \
  --header 'user_id: test' \
  --header 'session_id: local_session' \
  --data '{"prompt": "讲一个笑话！"}'


参数说明：
user_id：用户标识符
session_id：会话标识符
prompt：用户的自然语言请求

### 5. 查看 Agentkit 观测数据

调用完成后，可以在 Agentkit 控制台的[可观测](https://console.volcengine.com/agentkit/region:agentkit+cn-beijing/apmplus)界面查看可观测数据。包括 runtime 监控，模型监控，会话分析等功能。详情参考 [Agentkit 数据观测](https://www.volcengine.com/docs/86681/1873528?lang=zh)。以下为部分功能的效果图：

#### Runtime 监控

![Agentkit 监控面板截图](./images/observability-agentkit-overview.jpg)
*图1: Agentkit Runtime 概览*

![模型性能监控](./images/observability-agentkit-runtime.jpg)
*图2: Agentkit Runtime 详情*

#### 模型监控
![模型性能监控](./images/observability-agentkit-model.jpg)
*图3: Agentkit 模型监控*

#### 会话分析
![会话分析界面](./images/observability-agentkit-session.jpg)
*图4: Agentkit 会话分析*

![会话详情](./images/observability-agentkit-session-detail.jpg)
*图5: Agentkit 会话详情*

#### Trace分析
![Trace 分析](./images/observability-agentkit-trace.jpg)
*图6: Agentkit Trace 分析*

![Trace 详情](./images/observability-agentkit-trace-detail.jpg)
*图7: Agentkit Trace 详情*


 ## 总结

本教程带领您完成了在非 AgentKit Runtime 部署环境下开启可观测性的完整流程，主要内容包括：

1. **可观测性接入方式**：
   - 在代码中引入 APMPlusExporter 和 OpentelemetryTracer
   - 初始化 APMPlusExporter 实例并配置到 Agent 中
   - 通过环境变量或默认配置完成可观测数据的上报配置
    
2. **完整操作流程**：
   - 安装 AgentKit 依赖
   - 使用 `agentkit init` 初始化 Agent 项目
   - 修改代码，添加 APMPlusExporter 相关配置
   - 运行 Agent 服务
   - 通过 curl 测试调用 Agent


3. **可观测功能介绍**：
   - **Runtime 监控**：查看 Agent 运行状态、请求量、响应时间等基础指标
   - **模型监控**：分析模型调用性能、成本等关键数据
   - **会话分析**：深入了解用户与 Agent 的交互过程和对话历史
   - **Trace 分析**：通过分布式追踪技术，详细查看 Agent 执行流程中的每个环节


通过本教程，您已经掌握了在非 AgentKit Runtime 环境下（如本地部署）为 Agent 开启可观测性的基本配置和使用方法。如需更深入了解相关功能，可以参考 [Agentkit 数据观测](https://www.volcengine.com/docs/86681/1873528?lang=zh) 官方文档。

您也可以扩展此示例的 Agent 以实现更加复杂的 Agent 的可观测性。